## About the Problem:
Using the machine learning tools, we need to analyze the information about the passensgers of Titanic and predict which passenger has survived. This problem has been published by Kaggle and is widely used for learning basic concepts of Machine Learning

## Exploring The Data:

### Data Dictionary
![Imgur](https://i.imgur.com/bkNeXxE.png)

## Variable Notes
**pclass**: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...

**Sibling** = brother, sister, stepbrother, stepsister

**Spouse** = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...

**Parent** = mother, father

**Child** = daughter, son, stepdaughter, stepson

Some children travelled only with a nanny, therefore parch=0 for them.

## Load Libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnull, when, count, col
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pandas.plotting import scatter_matrix
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pandas as pd
import numpy as np

## Load Data

In [ ]:
spark = SparkSession.builder.appName('tps-2021-ml-with-pyspark').getOrCreate()
df = spark.read.csv(r'Y:\ML - DL - NLP - CV\Work Space_coding\Big Data Project Cifar 10\tabular-playground-series-apr-2021\train.csv', header = True, inferSchema = True)
df.printSchema()

## Show Dataset


In [ ]:
df.show(5)

Now, Data grouping by Survived for checking the classes are perfectly balanced!!

In [ ]:
df.groupby('Survived').count().show()

**Survived class not balanced!!**

## Summary statistics for numeric variables

In [ ]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().show()

## Correlations

Checking Correlations between independent variables


In [ ]:
numeric_data = df.select(numeric_features).toPandas()

axs = scatter_matrix(numeric_data, figsize=(8, 8));

# Rotate axis labels and remove axis ticks
n = len(numeric_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

## Data preparation and feature engineering

### 1. Missing Data Handling:


In [ ]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

In [ ]:
dataset = df.replace('null', None)\
        .dropna(how='any')

**Again check missing data**

In [ ]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

## 2. Unnecessary columns dropping


In [ ]:
dataset = dataset.drop('PassengerId')
dataset = dataset.drop('Ticket')
dataset = dataset.drop('Fare')
dataset = dataset.drop('Cabin')
dataset = dataset.drop('Embarked')
dataset = dataset.drop('Name')
dataset.show()

## 3. Features Convert into Vector

In [ ]:
sex_in = StringIndexer(inputCol="Sex", outputCol="Sex_encoding")
df1 = sex_in.fit(dataset).transform(dataset)
df1.show()

In [ ]:
required_features = ['Survived',
                    'Pclass',
                    'SibSp',
                    'Parch',
                     'Age',
                     'Sex_encoding'
                   ]

assembler = VectorAssembler(inputCols=required_features, outputCol='features')

transformed_data = assembler.transform(df1)

In [ ]:
transformed_data.show()

## 4. Train and Test Split
Randomly split data into train and test sets, and set seed for reproducibility.

In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [ ]:
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

# Machine learning Model Building


## 1. Random Forest Classifier

Random forest is a supervised learning algorithm which is used for both classification and regression cases, as well. But however, it is mainly used for classification problems. As we know that a forest is made up of trees and more trees mean more robust forests, in a similar way, random forest algorithm creates decision trees on data samples and then gets the prediction from each of them and finally selects the best solution by means of voting. It is an ensemble method which is better than a single decision tree because it reduces the over-fitting by averaging the result.

In [ ]:
rf = RandomForestClassifier(labelCol='Survived', 
                            featuresCol='features',
                            maxDepth=5)
model = rf.fit(training_data)
rf_predictions = model.transform(test_data)

**Evaluate our Random Forest Classifier model.**

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'Survived', metricName = 'accuracy')
print('Random Forest classifier Accuracy:', multi_evaluator.evaluate(rf_predictions))

## 2. Decision Tree Classifier

Decision trees are widely used since they are easy to interpret, handle categorical features, extend to the multiclass classification setting, while not requiring feature scaling and are able to capture non-linearities and feature interactions.

In [ ]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'Survived', maxDepth = 3)
dtModel = dt.fit(training_data)
dt_predictions = dtModel.transform(test_data)
dt_predictions.select('Sex_encoding', 'Pclass','SibSp', 'Parch', 'Age', 'Survived').show(10)

**Evaluate our Decision Tree model.**

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'Survived', metricName = 'accuracy')
print('Decision Tree Accuracy:', multi_evaluator.evaluate(dt_predictions))

## 3. Gradient-boosted Tree classifier Model

Gradient boosting is a machine learning technique for regression and classification problems, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees.

In [ ]:
gb = GBTClassifier(labelCol = 'Survived', featuresCol = 'features')
gbModel = gb.fit(training_data)
gb_predictions = gbModel.transform(test_data)

**Evaluate our Gradient-Boosted Tree Classifier.**

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'Survived', metricName = 'accuracy')
print('Gradient-boosted Trees Accuracy:', multi_evaluator.evaluate(gb_predictions))

# Make Sumbission file

In [ ]:
df_pd = gb_predictions.toPandas()

In [ ]:
df_pd.head()

In [ ]:
df_pd.to_csv("gbModel.csv",index = False)

In [ ]:
gb_df = pd.read_csv("./gbModel.csv")

In [ ]:
gb_df.head()

In [ ]:
submission  = pd.read_csv(r"Y:\ML - DL - NLP - CV\Work Space_coding\Big Data Project Cifar 10\tabular-playground-series-apr-2021\sample_submission.csv")
submission = submission.drop("Survived",axis=1)
submission.head()

In [ ]:
results = pd.concat([submission,gb_df.Survived],axis = 1)

In [ ]:
results.to_csv("submission.csv", index = False)